# Tutorial 2: Add MinIO to an existing VM (optional)

### **Requirements:** a VM running your MySQL database (tutorial 1 complete).


In this tutorial we'll cover how to launch MinIO from an existing VM (which was configured in the previous tutorial). 

MinIO is an object storage server useful for storing large files. If all of your data files are small (roughly <10 MB), you can store everything directly in the MySQL database and skip this part. If you need to access larger files (and have the storage for them), then this tutorial is for you.

-----------------------

## 1. Create the `minio` directory

First, create a folder on your VM to store your bulky data in (we'll assume that the folder is called `minio`). 

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
sudo mkdir minio
</pre>

You might have to update permissions to use this folder:

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
sudo chown ubuntu: minio
</pre>

## 2. Mount your external volume

The specifics here will vary based on your setup. We'll cover some general steps, but if you're part of an organization, you should probably reach out to a system admin.

If you don't have enough storage on the VM itself and are attaching an extra storage volume, then attach the volume to the `minio` folder (e.g. via your cloud computing platform, such as OpenStack, or by plugging in an external hard drive and making sure it's recognized on the VM). Run `lsblk` or `df -h` to list all devices and check that it is attached (it will probably look something like `/dev/sdb1`). 

## 3. Launch the MinIO server

We'll make a new terminal session using tmux, a terminal multiplexer (which is a fancy way of saying that it opens terminal sessions which you can attach to or detatch from). We'll launch our MinIO service in this terminal. This ensures that MinIO will be running in this terminal even if you disconnect from the server or your network connection is interrupted. 

Let's create a new terminal session and name it `minio`:

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
tmux new -s minio
</pre>

Now you're in your new tmux terminal. 

(You can detach from the tmux session and return to your normal terminal by pressing `Ctrl+B D`. You can later reattach to the session using the command `tmux attach -t minio`.)

Now we'll launch our MinIO server container using Docker. The command below maps ports 9000 and 9001 from the host to the container for server and console access, mounts the host `minio` directory as a volume for data storage, and sets up root access credentials for the server. **Enter a password of your choosing.**

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
sudo docker run -p 9000:9000 -p 9001:9001 --name minio -v ~/minio:/data -e "MINIO_ROOT_USER=root" -e "MINIO_ROOT_PASSWORD=&lt;password&gt;" quay.io/minio/minio:RELEASE.2021-10-23T03-28-24Z server /data -console-address ":9001"
</pre>

You should now be able to access your MinIO console via a web browser at `http://<your.ip>:9001`.

## 4. Launch the MinIO client

Let's first split the current terminal pane vertically: `Ctrl+B Shift+%`

Now, from our new pane, we'll launch the MinIO client:

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
sudo docker run --name minio1_client --net=host -it -v ~/minio:/data --entrypoint=/bin/sh minio/mc
</pre>

The above command again binds the local directory `~/minio` to the `/data` directory inside the container. 

Now we'll tell the MinIO client to remember our new MinIO server and make it accesible via browser. **Enter your password of choice.**  

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
mc config host add minio http://127.0.0.1:9001 root "&lt;password&gt;"
</pre>

Now you're ready to store data on MinIO!

You can access the MinIO client from your browser at `http://127.0.0.1:9001`.

In tutorial 5 we'll cover how to populate MinIO with your bulky data.